# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
The following project is the first project of the Udacity- Self-Driving Car Engineer Nanodegree. The aim of this project was to implement a pipeline to detect lanes on the road. The pipeline was first tested on a series of images and then on a video stream. 

The following pipeline was used to obtain a satisfactory and annotated video of lanes on an highway.

    1. Image is read using OpenCV
    2. The image is converted in Grayscale
    3. Gausian Filter is applied on the grayscale image.
    4. Canny edge detection is performed on the image.
    5. A region of interest is selected and a mask is created from it.
    6. Using hough transform lines are detected on the image.
    7. The lines obtained are interpolated to mark the lanes properly.
    8. The pipeline for single image is applied on a video which is just series of images.
       


<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> This is the result without interpolation of the lines </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> This is the final desired result.</p> 
 </figcaption>
</figure>

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2
%matplotlib inline

In [2]:
# Function to convert the image to grayscale
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
# Function to apply the Canny transform
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

# Function to apply the gaussian blur
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

# Function to obtain the vertices for the region of interest.
def vertices(img):
    x=img.shape[1]
    y=img.shape[0]
    LThres=50
    T = 80 # parameter for edges of trapezium
    vertices = np.array([[(LThres,y),(x/2-T, y/2+T), (x/2+T, y/2+T), (x-LThres,y)]], dtype=np.int32)
    return vertices

# Function to obtain the region of interest
def region_of_interest(img, vertices):
   
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
        
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=((y2-y1)/(x2-x1)) 
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

# Modified function to interpolate the lines obtained            
def draw_lines2(img, lines, color=[255, 0, 0], thickness=2):
       
    #algorithm logic:
    #aim is to find x_min, y_min, x_max, y_max , slope and intercept for both lanes lines.
    #for each line returned from the hough lines function:
    #   calculate slope
    #   calculate intercept
    #   store positive and negative slope and intercept values separately in arrays.
    #   y_min is the minimum of all the y coordinates.
    #   y_max is the bottom of the image from where the lane lines start.
    #   slope and intercept values for both lines are just the averages of all values stored previously.
    #  x_min and x_max can now be calculated by fitting all the lines in the equation x = (y - intercept)/slope.
    
    #LINE DISPLAY PARAMETERS
    color = [255, 0, 0]
    thickness = 12
    
    #LINE PARAMETERS
    SLOPE_THRESHOLD = 0.3
    Y_MIN_ADJUST = 15
    
    positive_slopes = []
    negative_slopes = []
    
    positive_intercepts = []
    negative_intercepts = []
    
    #named as y_max despte being at the bottom corner of the image due to y axis in reverse direction
    y_max = img.shape[0]
    y_min = img.shape[0]
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            #calculate slope for the line
            slope = (y2-y1)/(x2-x1)
            intercept = y2 - (slope*x2)
            
            #for negative slope
            if slope < 0.0 and slope > -math.inf and abs(slope) > SLOPE_THRESHOLD:
                #print('negative slope')
                negative_slopes.append(slope)
                negative_intercepts.append(intercept)
                
            #for positive slope
            elif slope > 0.0 and slope < math.inf and abs(slope) > SLOPE_THRESHOLD:
                #print('positive slope')
                positive_slopes.append(slope)
                positive_intercepts.append(intercept)
            
            y_min = min(y_min, y1, y2)
    
    y_min+=Y_MIN_ADJUST
    
    #get averages for positive and negative slopes
    positive_slope_mean = np.mean(positive_slopes)
    negative_slope_mean = np.mean(negative_slopes)

    #get averages for potitive and negative intercepts
    positive_intercept_mean = np.mean(positive_intercepts)
    negative_intercept_mean = np.mean(negative_intercepts)
    
    #calculation of coordinates for lane for positive slopes
    if len(positive_slopes) > 0:
        x_max = int((y_max - positive_intercept_mean)/positive_slope_mean)
        x_min = int((y_min - positive_intercept_mean)/positive_slope_mean)
        cv2.line(img, (x_min, y_min), (x_max, y_max), color, thickness)
    
    #calculation of coordinates for lane for negative slopes
    if len(negative_slopes) > 0:
        x_max = int((y_max - negative_intercept_mean)/negative_slope_mean)
        x_min = int((y_min - negative_intercept_mean)/negative_slope_mean)
        cv2.line(img, (x_min, y_min), (x_max, y_max), color, thickness)

        
def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    slopes = [] 
    intercepts = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=((y2-y1)/(x2-x1)) 
            slopes.append(slope)
            intercept = y2 - (slope*x2)
            intercepts.append(intercept)
            
    for index in range(0, len(slopes)-1):
        totalSlope = slopes[i] + slopes[i+1] + slopes[i+2]
        runningAvgSlope = totalSlope/3
        
        totalIntercept = slopes[i] + slopes[i+1] + slopes[i+2]
        runningAvgIntercept = totalIntercept/3
        
        
#  Returns an image with hough lines drawn.
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    # print(type(lines))
    # print(lines.shape[::])
    # print(lines[0])
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines2(line_img, lines)
    return line_img

# Function to produce weighted image of the original image and the lines obtained.
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [3]:
# Creating a list of images for easy of use later.
import os
a=os.listdir("test_images/")

## Build a Lane Finding Pipeline



In [4]:
# The pipeline that is explained above.
def pipeline(img):
    gray = grayscale(img)
    gaus = gaussian_blur(gray,5)
    c = canny(gaus, 120, 200)
    roi=region_of_interest(c,vertices(img))
    rho = 3            # distance resolution in pixels of the Hough grid
    theta = np.pi/180  # angular resolution in radians of the Hough grid
    threshold = 40     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 70  # minimum number of pixels making up a line
    max_line_gap = 100       
    h = hough_lines(roi, rho, theta, threshold, min_line_len, max_line_gap)
    final = weighted_img( h,img, α=0.8, β=1., γ=0.)
    return final
    
    
for i in range(6):
    image = cv2.imread("test_images/"+a[i])
    final = pipeline(image)
    cv2.imwrite("test_images_output/"+a[i]+".jpg",final)
  


## Test on Videos

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(white_output, audio=False)

t:   9%|▉         | 20/221 [00:00<00:01, 194.89it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
CPU times: user 5.89 s, sys: 125 ms, total: 6.02 s
Wall time: 1.93 s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left.

In [8]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(pipeline)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   0%|          | 0/681 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
CPU times: user 19.2 s, sys: 676 ms, total: 19.9 s
Wall time: 6.66 s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

The following video has sudden bright window where the detection of lanes using current algorithm is difficult. However using several other methods like playing with colorspaces etc. we can detect the lanes in this video quite easily.

In [10]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(pipeline)
%time challenge_clip.write_videofile(challenge_output, audio=False)

t:   0%|          | 0/251 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/challenge.mp4
CPU times: user 9.99 s, sys: 528 ms, total: 10.5 s
Wall time: 4.89 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))